# DNN tuning techniques

In [ ]:
# Importing the libraries
from tensorflow.keras import models as k_models
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
# Load the dataset (Iris from sklearn)
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# define the constants 
feature_dim = X_train.shape[1]
target_dim = 1

In [ ]:
# Tune DNN Using GridSearchCV

param_grid = {
    'batch_size': [10, 20, 40],
    'epochs': [10, 50, 100],
    'optimizer': ['SGD', 'Adam'],
    'init': ['uniform', 'normal'],
    'units': [32, 64, 128],
}

def create_model(optimizer='adam', init='uniform'):
    _model = k_models.Sequential([
        Input(shape=(feature_dim,)),
        Dense(12, kernel_initializer=init, activation='relu'),
        Dense(8, kernel_initializer=init, activation='relu'),
        Dense(target_dim, kernel_initializer=init, activation='sigmoid')
    ])
    _model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return _model

model = KerasClassifier(build_fn=create_model, verbose=0)

# Perform Grid Search:  
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# best model
best_model = grid_result.best_estimator_.model
best_params = grid_result.best_params_

# Summarize the Results:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print(f"{mean} ({std}) with: {param}")

In [ ]:
# Tune DNN Using keras-tuner

def build_model(hp: kt.HyperParameters):
    _model = keras.Sequential([
        Input(shape=(feature_dim,)),
        layers.Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32), activation='relu'),
        layers.Dense(units=hp.Int('units_2', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(target_dim, activation='sigmoid')
    ])
    
    # Tune the learning rate
    _model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
# Run the Hyperparameter Search:  
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='intro_to_kt')

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_0')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")